In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import xarray as xr
from matplotlib import gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import ticker
from datetime import datetime, timedelta, date
from matplotlib.path import Path
import matplotlib.patheffects as pe
import glob,os
import time
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
import scipy
from scipy import stats

In [6]:
###parameters###
startyear,startmonth,startday=2022,7,1
endyear,endmonth,endday=2022,8,1

email='bwallace3@albany.edu' #this needs to be substituted with your own email that you used to signup for the API
key='greycat81' #once you've signed up with your email, they'll assign a personal key to you

params='44201,42101,42602,42603,88101' #refer to paramaters.csv or https://aqs.epa.gov/aqsweb/documents/codetables/parameters.html
                                       #44201 - O3, 42101 - CO, 42602 - NO2, 42603 - NO, 88101 - PM2.5
state='36'
county='081'
site='0124'

dates=np.arange(datetime(startyear,startmonth,startday),datetime(endyear,endmonth,endday),timedelta(days=1)).astype(datetime)

In [7]:
for day in dates:
    dyr=day.year
    dmo=day.month
    ddy=day.day
    sdyr,sdmo,sddy=str(dyr),str(dmo).zfill(2),str(ddy).zfill(2)
    
    
    pdobj=pd.read_json('https://aqs.epa.gov/data/api/sampleData/bySite?email={0}&key={1}&param={2},&bdate={3}&edate={3}&state={4}&county={5}&site={6}'.format(
        email,key,params,sdyr+sdmo+sddy,state,county,site),orient='index')
    
    df=pd.concat([pd.json_normalize(pdobj[col_idx].Data) for col_idx in pdobj.columns])

    times=df.date_local+'_'+df.time_local
    times=pd.to_datetime(times,format='%Y-%m-%d_%H:%M')

    df=df.set_index(times).sort_index()
    df.index.names=['date']

    combined_df=pd.concat([df[(df.parameter_code==code)&(df.sample_frequency=='HOURLY')].sample_measurement.rename(df[df.parameter_code==code].parameter[0]) for code in df.parameter_code.unique()],axis=1)

    #convert CO and O3 to PPB to be consistent with PM2.5, NO2, & NO
    combined_df['Carbon monoxide']=combined_df['Carbon monoxide']*1000
    combined_df['Ozone']=combined_df['Ozone']*1000
    
    combined_df.to_csv('/home/bwallace/EPA_AQS/{0}.csv'.format(sdyr+sdmo+sddy))

NotImplementedError: 